In [31]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression

Рассмотрим задачу множественной регрессии. Допустим, у нас есть дата сет, в котором указан рост абитуриента, суммарный балл, который он получил/ получила на ЕГЭ , примерное количество часов, которое было потрачено на подготовку к экзаменам, и стоимость обучения в ВУЗЕ. Логично предположить наличие обратной зависимости цены обучения от количества часов и суммарного балла (у абитуриентов с высокими баллами бывают скидки на обучение, а некоторые даже учатся бесплатно). И предположим "ненастоящую зависимость" от роста абитуриента (вдруг, чем выше абитуриент, тем дешевле ему учиться ;) )

In [32]:
# Создаём сэмпл
n_samples = 600 # количество наблюдений / объектов

height_owner = np.random.choice(50, n_samples) + 150 #рост абитуриента
sum_ege = np.random.choice(180, n_samples) + 120 #суммарный балл егэ
col_hours = np.random.choice(500, n_samples) + 100 #количество часов, потраченных на подготовку к егэ

price = 300000 - sum_ege * col_hours   #стоимость обучения в вузе в семестр примерно, формула искусственна по сути

data = pd.DataFrame({'height_owner': height_owner, 'sum_ege': sum_ege, 'col_hours': col_hours, 'price': price})
data.head(5)

,height_owner,sum_ege,col_hours,price
0,151,268,554,151528
1,156,165,262,256770
2,183,204,491,199836
3,171,234,369,213654
4,179,182,349,236482


Попробуем сделать первую тестовую модель, в которой предположим зависимость стоимости обучения от роста,
суммы баллов и количества часов

In [33]:
from sklearn.metrics import mean_absolute_error

X = data[['height_owner', 'sum_ege', 'col_hours']]
y = data['price']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['height_owner', 'sum_ege', 'col_hours']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))


Weights: [   4.9230212  -346.47618363 -208.5227684 ]
Bias: 371567.10884934146
Error: 5700.911221065996


ожидаемая стоимость обучения

In [34]:
y.median()

233307.0

Получаем ошибку в 6 тысяч, биас 372 тысячи, это как бы многовато и показывает то, что построенная модель не совсем адекватна и не совсем отражает зависимость
Попробуем теперь еще раз построить модель линейной регрессии, уберем рост. Посмотрим, есть ли регрессия между стоимостью и суммой баллов, а также количеством часов 

In [35]:
X = data[['sum_ege', 'col_hours']]
y = data['price']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))  # веса, коэффициенты регрессии
print('Bias: {}'.format(reg.intercept_)) # 

pred_values = reg.predict(data[['sum_ege', 'col_hours']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [-346.48460889 -208.53725685]
Bias: 372427.842726201
Error: 5701.954947427251


Судя по коэффициентам модели, по оценке ошибке, данная модель настолько же плоха, как и первая. Создадим новую переменную, и построим регрессию с ее помощью.

In [40]:
# Создаем новый признак
data['cost'] = 200000 - data['sum_ege'] * data['col_hours']
data.head(5)

,height_owner,sum_ege,col_hours,price,cost
0,151,268,554,151528,51528
1,156,165,262,256770,156770
2,183,204,491,199836,99836
3,171,234,369,213654,113654
4,179,182,349,236482,136482


In [41]:
data['cost'].median()

133307.0

In [42]:
X = data[['cost']]
y = data['price']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['cost']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [1.]
Bias: 99999.99999999996
Error: 6.2088171641031905e-12


Видим, что в последней модели ошибка меньше на несколько порядков (правда и коэффициент регрессии стал равен единице))) ) но таким образом, можно сделать вывод, что мы научились моделировать с помощью метода множественной линейной регрессии, а также корректировать модель на основе получающихся данных, учитывать возникающую ошибку при моделировании.